# Training and Tuning at Scale

#### Amazon SageMaker provides managed distributed training and tuning capabilities to improve training efficiency, and capabilities to organize and track ML experiments at scale. SageMaker enables techniques such as streaming data into algorithms by using pipe mode for training with data at scale and Managed Spot Training for reduced training costs. 

## ML training at scale with SageMaker distributed libraries

#### Two common scale challenges with ML projects are scaling training data and scaling model size. While increased training data volume, model size, and complexity can potentially result in a more accurate model, there is a limit to the data volume and the model size that you can use with a single compute node, CPU, or GPU. Increased training data volumes and model sizes typically result in more computations, and therefore training jobs take longer to finish, even when using powerful compute instances such as Amazon Elastic Compute Cloud (EC2) p3 and p4 instances.

#### Distributed training is a commonly used technique to speed up training when dealing with scale challenges. Training load can be distributed either across multiple compute instances (nodes), or across multiple CPUs and GPUs (devices) on a single compute instance. There are two strategies for distributed training – data parallelism and model parallelism. Their names are a good indication of what is involved with each strategy. With data parallelism, the training data is split up across multiple nodes (or devices). With model parallelism, the model is split up across the nodes (or devices).

#### Mixed-precision training is a popular technique to handle training at scale and reduce training time. Typically used on compute instances equipped with NVIDIA GPUs, mixed-precision training converts network weights from FP32 representation to FP16, calculates the gradients, converts weights back to FP32, multiplies by the learning rate, and finally updates the optimizer weights.

#### In the data parallelism distribution strategy, the ML algorithm or the neural network-based model is replicated on all devices, and each device processes a batch of data. Results from all devices are then combined. In the model parallelism distribution strategy, the model (which is the neural network) is split up across the devices. Batches of training data are sent to all devices so that the data can be processed by all parts of the model. 

#### Both data and model parallelism distribution strategies come with their own complexities. With data parallelism, each node (or device) is trained on a subset of data (called a mini-batch), and a mini-gradient is calculated. However, within each node, a mini-gradient average, with gradients coming from other nodes, should be calculated and communicated to all other nodes. This step is called all reduce, which is a communication overhead that grows as the training cluster is scaled up.

#### While model parallelism addresses the requirements of a model not fitting in a single device's memory by splitting it across devices, partitioning the model across multiple GPUs may lead to under-utilization. This is because training on GPUs is sequential in nature, where only one GPU is actively processing data while the other GPUs are waiting to be activated. To be effective, model parallelism should be coupled with a pipeline execution schedule to train the model across multiple nodes, and in turn, maximize GPU utilization. Now that you know two different distribution strategies, how do you choose between data and model parallelism?

## Choosing between data and model parallelism

##### When choosing a distributed strategy to implement, keep in mind the following:

- Training on multiple nodes inherently causes inter-node communication overhead.
- Additionally, to meet security and regulatory requirements, you may choose to protect the data transmitted between the nodes by enabling inter-container encryption.
- Enabling inter-container encryption will further increase the training time.


#### Due to these reasons, use data parallelism if the trained model can fit in the memory of a single device or node. In situations where the model does not fit in the memory due to its size or complexity, you should experiment further with data parallelism before deciding on model parallelism.

- Tuning the model's hyperparameters: Tuning parameters such as the number of layers of a neural network, or the optimizer to use, affects the model's size considerably.
- Reducing the batch size: Experiment by incrementally reducing the batch size until the model fits in the memory. This experiment should balance out the model's memory needs with optimal batch size. Make sure you do not end up with a suboptimal small batch size just because training with a large batch size takes up most of the device memory.
- Reducing the model input size: If the model input is tabular, consider embedding vectors of reduced dimensions. Similarly, for natural language processing (NLP) models, reduce the input NLP sequence length, and if the input is an image, reduce image resolution.
- Using mixed-point precision: Experiment with mixed-precision training, which uses FP16 representation of weights during gradient calculation, to reduce memory consumption.
